## Diabetes

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import numpy as np

device = torch.device('cpu')

In [4]:
from sklearn.datasets import load_diabetes

data = load_diabetes()
print (data.feature_names)

X, y = data.data, data.target

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']


# 1. Specifying input and targets

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.33, random_state = 42)

In [28]:
# change numpy to torch
train_input  = torch.tensor (X_train, dtype = torch.float32)
test_input   = torch.tensor (X_test, dtype = torch.float32)
train_output = torch.tensor (y_train, dtype = torch.float32)
test_output  = torch.tensor (y_test, dtype = torch.float32)

In [7]:
train_input.shape, test_input.shape, train_output.shape, test_output.shape

(torch.Size([296, 10]),
 torch.Size([146, 10]),
 torch.Size([296]),
 torch.Size([146]))

In [25]:
train_input.dtype, test_input.dtype, train_output.dtype, test_output.dtype

(torch.float64, torch.float64, torch.float64, torch.float64)

# 2. Dataloaders

In [8]:
print (data.feature_names)

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']


In [9]:
train_ds = TensorDataset (train_input, train_output)
test_ds  = TensorDataset (test_input, test_output)
# train_ds[0]

In [10]:
print(f"",len(train_ds))
296 / 32
'''

296 / 32 = 10 rounds
'''


 296


'\n\n296 / 32 = 10 rounds\n'

In [11]:
batch_size = 32 # binary 
train_dl = DataLoader (train_ds, batch_size = batch_size, shuffle = True,  num_workers = 4)
test_dl  = DataLoader (test_ds,  batch_size = batch_size, shuffle = True, num_workers = 4)

In [12]:
# # test
# for batch_x, batch_y in train_dl:
#     print ("One batch of X:", batch_x.shape)
#     print ("One batch of y:", batch_y.shape)
#     break

# 3. Layers

In [13]:
model = nn.Sequential(
    nn.Linear (10, 24), # 10 features in the dataset
    nn.ReLU(), # gradient = 1
    # nn.Sigmoid () >> allow us to learn non-linear funciton - activation function
    nn.Linear (24, 12),
    nn.ReLU(), # you can use this instead of Sigmoid: tanh, relu, leakly, swinrelu, etc.
    nn.Linear (12, 6),
    nn.ReLU(),
    nn.Linear (6, 1)
)

# any number of matrix mautiplication can be approximate into one matrix multiplication

In [18]:
total_num_of_params = 0
for ix, p in enumerate(model.parameters()):
    # print (p.shape)
    print (p.numel()) # total number in layer: [10, 24] = 240 | bias = 24
    total_num_of_params += p.numel()
print (total_num_of_params)

240
24
288
12
72
6
6
1
649


In [23]:
# train_input.dtype

torch.float64

In [30]:
# always good to test your neural network before training
yhat = model(train_input)
yhat.shape
assert yhat.shape[1] == 1

# 4. Loss function

# 5. Training

# 6. Testing